In [ ]:
## 4.1 準備
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
## 4.2 インデックスの構築
"""ドキュメントを読み込んでインデックスを構築します。
importしているのは、CLASS
https://python.langchain.com/docs/how_to/document_loader_directory/
"""
from langchain_community.document_loaders import DirectoryLoader
""" https://python.langchain.com/docs/integrations/document_loaders/pypdfloader/#loader-features """
from langchain_community.document_loaders import PyPDFLoader

# PDFファイルを読込
"""
ドキュメントの読込にはDocument loaderを使用します。今回はフォルダの中にあるファイルのテキスト情報を読み込む DirectoryLoader を使用します。
DirectoryLoader() のパラメータは以下のとおりです。
・ './data/pdf' ：読み込み先のフォルダ
・ glob="./*.pdf" ：読み込む対象のファイル（今回はPDF）
・ loader_cls=PyPDFLoader ：ファイルの読み込みに使用するDocument loader
"""
loader = DirectoryLoader('./data/pdf', glob="./*.pdf",   loader_cls=PyPDFLoader)
documents = loader.load()

# 結果の表示
print(documents)


[Document(metadata={'source': 'data/pdf/02賃金規則.pdf', 'page': 0}, page_content='みらいテクノロジー株式会社  賃⾦規則\nみらいテクノロジー株式会社では、従業員の皆さんが安⼼して働けるよう、賃⾦（給与）に関す\nるルールを明確に定めています。この賃⾦規則は、給与の構成や⽀払い⽅法、昇給や賞与の仕組\nみを理解し、働きがいを持って業務に取り組んでもらうために作られています。\n1. 基本給\n1. 基本給とは\n基本給は、みらいテクノロジー株式会社で働くすべての従業員に⽀払われる基本的\nな賃⾦です。\n基本給は、従業員の経験や能⼒、職務内容に基づいて決定されます。\n2. 基本給の決定⽅法\n基本給は、毎年の⼈事評価結果や会社の業績、個⼈の勤務年数などを考慮して⾒直\nしが⾏われる場合があります。\n新⼊社員の場合は、会社の定める給与テーブルに基づき、職種ごとに⼀定の基準額\nが設定されています。\n2. 各種⼿当\nみらいテクノロジー株式会社では、基本給に加えて、以下の⼿当が⽀給されます。\n1. 通勤⼿当\n通勤にかかる交通費は、実際の経路に基づき⽀給されます。\n公共交通機関の利⽤の場合は、最安経路をもとに⽉額上限 3 万円まで⽀給します。\n⾃家⽤⾞での通勤が必要な場合は、事前に⼈事部へ申請してください。駐⾞場の使\n⽤料やガソリン代の⼀部が⽀給される場合もあります。\n2. 住宅⼿当\n会社から通勤に 1 時間以上かかる場合、住宅⼿当として⽉額 1 万円が⽀給されます。\n住宅⼿当を受けるためには、賃貸契約書など、居住地を証明できる書類の提出が必\n要です。\n3. 家族⼿当\n扶養家族がいる従業員には、家族⼿当が⽀給されます。\n配偶者には⽉額 5,000 円、⼦供⼀⼈につき⽉額 3,000 円が⽀給されます（上限︓⼦供 3\n⼈まで）。\n4. 時間外⼿当（残業代）\n所定の勤務時間を超えて働いた場合は、残業⼿当が⽀給されます。\n残業⼿当の割増率は、法令に基づき計算されます。通常の時間外労働は 1.25 倍、深\n夜時間帯（午後 10 時以降）の残業は 1.5 倍となります。\n5. 休⽇出勤⼿当'), Document(metadata={'source': 'data/

In [4]:
# 視認性の高い Document リストの可視化ユーティリティ
# - Rich の表で見やすく整形（未インストールなら自動インストール）
# - 1件ずつ ソース/ページ/本文スニペット を一覧表示
# - インデックス指定で詳細（全文 or 先頭N文字）も確認可能

# 依存ライブラリの用意
try:
    from rich.console import Console
    from rich.table import Table
    from rich.panel import Panel
    from rich.text import Text
except ImportError:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "rich"])
    from rich.console import Console
    from rich.table import Table
    from rich.panel import Panel
    from rich.text import Text

from textwrap import shorten, fill

console = Console()

# documents が存在するかチェック
if 'documents' not in globals():
    raise NameError("documents が未定義です。print(documents) を実行できたノートブック（セル）の直後で実行してください。")

def preview_documents(docs, max_chars=220, wrap=60, show=50):
    """
    docs: list-like of Document (langchainなどの Document を想定)
    max_chars: 本文スニペットの最大文字数
    wrap: スニペットの折り返し幅（Noneで折り返しなし）
    show: 先頭から何件表示するか（多すぎると重いので上限つき）
    """
    if not docs:
        console.print("[bold yellow]documents は空です[/]")
        return

    n = min(len(docs), show)
    table = Table(title=f"Documents preview  (showing {n}/{len(docs)})", show_lines=False)
    table.add_column("#", style="cyan", no_wrap=True)
    table.add_column("source", style="green")
    table.add_column("page", style="magenta", justify="right", no_wrap=True)
    table.add_column("content (snippet)", style="white")

    for i, doc in enumerate(docs[:n]):
        # Document 互換：.metadata, .page_content がない場合のフォールバック
        meta = getattr(doc, "metadata", {}) or {}
        content = getattr(doc, "page_content", str(doc)) or ""

        source = str(meta.get("source", ""))
        page = str(meta.get("page", ""))

        snippet = content.replace("\r", " ").replace("\n", " ")
        snippet = shorten(snippet, width=max_chars, placeholder=" ...")
        if wrap:
            snippet = fill(snippet, width=wrap)

        table.add_row(str(i), source, page, snippet)

    console.print(table)

def show_document_detail(docs, idx=0, max_chars=None, wrap=80):
    """
    個別ドキュメントの詳細を確認
    idx: 確認したいインデックス（preview の # と同じ）
    max_chars: Noneなら全文表示。数値なら先頭からその文字数まで
    wrap: 折り返し幅
    """
    if idx < 0 or idx >= len(docs):
        raise IndexError(f"idx={idx} は範囲外です（0..{len(docs)-1}）")

    doc = docs[idx]
    meta = getattr(doc, "metadata", {}) or {}
    content = getattr(doc, "page_content", str(doc)) or ""
    if max_chars is not None:
        content = content[:max_chars]

    if wrap:
        content_wrapped = fill(content, width=wrap)
    else:
        content_wrapped = content

    header = Text(f"Document #{idx}", style="bold cyan")
    meta_lines = [f"{k}: {v}" for k, v in meta.items()]
    meta_text = "\n".join(meta_lines) if meta_lines else "(no metadata)"
    panel = Panel.fit(
        Text.from_markup(
            f"[bold green]source[/]: {meta.get('source','')}\n"
            f"[bold magenta]page[/]: {meta.get('page','')}\n"
            f"[bold blue]metadata[/]:\n{meta_text}\n\n"
            f"[bold white]content[/]:\n{content_wrapped}"
        ),
        title=header,
        border_style="cyan"
    )
    console.print(panel)

# ===== 使い方 =====
# 一覧プレビュー（先頭50件・本文スニペット220文字・折り返し60桁）
preview_documents(documents, max_chars=220, wrap=60, show=50)

# 個別詳細（例：0番を全文折り返し80桁で表示）
# show_document_detail(documents, idx=0, max_chars=None, wrap=80)

# 個別詳細（例：3番の先頭1000文字だけを見る）
# show_document_detail(documents, idx=3, max_chars=1000, wrap=80)


                                        Documents preview  (showing 17/17)                                         
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ #  ┃ source                            ┃ page ┃ content (snippet)                                               ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0  │ data/pdf/02賃金規則.pdf           │    0 │ みらいテクノロジー株式会社 賃⾦規則                             │
│    │                                   │      │ みらいテクノロジー株式会社では、従業員の皆さんが安⼼して働ける… │
│    │                                   │      │ るルールを明確に定めています。この賃⾦規則は、給与の構成や⽀払… │
│    │                                   │      │ みを理解し、働きがいを持って業務に取り組んでもらうために作られ… │
│    │                                   │      │ 1. 基本給 1. 基本給とは                                         │
│    │                                   │      │ 基本給は、みらいテクノロジー株式会社で働くすべての従業員に⽀払… │
│    │                                   │      │ な賃⾦です。 ...                                                │
│ 1  │ data/pdf/02賃金規則.pdf           │    1 │ 休⽇に出勤する場合には、休⽇出勤⼿当が⽀給されます。            │
│    │                                   │      │ 休⽇出勤⼿当の割増率は 1.35 倍です。 3. 賞与（ボーナス）        │
│    │                                   │      │ 1. 賞与の⽀給基準 みらいテクノロジー株式会社では、年 2 回（ 6   │
│    │                                   │      │ ⽉と 12 ⽉）に賞与を⽀給します。                                │
│    │                                   │      │ 賞与の⾦額は、会社の業績や個⼈の評価結果に基づき決定されます。… │
│    │                                   │      │ 場合は、賞与の額が増えることもあります。                        │
│    │                                   │      │ 2. 賞与の計算⽅法 ...                                           │
│ 2  │ data/pdf/02賃金規則.pdf           │    2 │ 社会保険料や所得税、住⺠税など、法令で定められた控除が毎⽉の給… │
│    │                                   │      │ かれます。                                                      │
│    │                                   │      │ 控除内容について質問がある場合は、給与明細を確認し、不明点があ… │
│    │                                   │      │ 相談してください。 6. 退職⾦ 1.                                 │
│    │                                   │      │ 退職⾦の⽀給条件 みらいテクノロジー株式会社では、勤続 3         │
│    │                                   │      │ 年以上の従業員に対して退職⾦を⽀給し ます。                     │
│    │                                   │      │ 退職⾦は、退職時に⼀時⾦として⽀給されます。 2.                 │
│    │                                   │      │ 退職⾦の計算⽅法 ...                                            │
│ 3  │ data/pdf/05行動規範・倫理規程.pdf │    0 │ みらいテクノロジー株式会社 ⾏動規範・倫理規程                   │
│    │                                   │      │ みらいテクノロジー株式会社では、従業員⼀⼈ひとりが誠実かつ公平… │
│    │                                   │      │ 会的な責任を守るために「⾏動規範・倫理規程」を定めています。こ… │
│    │                                   │      │ 準や仕事上の倫理を明確にし、従業員が安⼼して働ける環境を作るた… │
│    │                                   │      │ 1. 基本的な⾏動指針 1. 誠実な⾏動                               │
│    │                                   │      │ 会社の利益を第⼀に考え、業務を進める際には誠実で正直な⾏動を⼼… │
│    │                                   │      │ ...                                                             │
│ 4  │ data/pdf/05行動規範・倫理規程.pdf │    1 │ 業務で扱う顧客情報や、会社の内部情報などは重要な財産です。これ… │
│    │                                   │      │ 三者に漏らすことは、厳しく禁じられています。                    │
│    │                                   │      │ 機密情報を含む資料やデータは、関係者以外に閲覧されないよう管理… │
│    │                                   │      │ い。 2. 個⼈情報の取り扱い                                      │
│    │                                   │      │ 従業員や顧客の個⼈情報は、法令および社内規定に基づき適切に保護… │
│    │                                   │      │ 個⼈情報の利⽤⽬的を明確にし、⽬的外の使⽤は⾏わないようにして… │
│    │                                   │      │ 3. SNS や外部メディアでの発⾔ 個⼈の                            │
│    │                                   │      │ SNS ...                                                         │
│ 5  │ data/pdf/05行動規範・倫理規程.pdf │    2 │ 3. 会社の名誉と信⽤の保護       

In [ ]:
# チャンクに分割
"""チャンクへの分割にはText Splitterを使用します。一般的には文字単位でテキストを分割する CharacterTextSplitter を使用することが多いでしょう。"""
from langchain_text_splitters import CharacterTextSplitter
import tiktoken

# 言語モデルに合うトークナイザー名を取得
"""tiktoken.encoding_for_model(MODEL_NAME).name で、言語モデルに合うトークナイザー名を取得しています。"""
encoding_name = tiktoken.encoding_for_model(MODEL_NAME).name

# テキスト分割を作成
"""
CharacterTextSplitter.from_tiktoken_encoder(encoding_name) で、Text Splitterを作成します。
区切り文字 separator や、チャンクサイズ chunk_size などのパラメータの指定も可能です。
"""
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(encoding_name)

# チャンクに分割
"""text_splitter.split_documents(documents) で、Text Splitterを使用してチャンクに分割します。"""
texts = text_splitter.split_documents(documents)

# チャンク数と内容の表示
print("texts_size=", len(texts))
for txt in texts[:3]:
    print(txt)
    print("-" * 50)

texts_size= 17
page_content='みらいテクノロジー株式会社  賃⾦規則
みらいテクノロジー株式会社では、従業員の皆さんが安⼼して働けるよう、賃⾦（給与）に関す
るルールを明確に定めています。この賃⾦規則は、給与の構成や⽀払い⽅法、昇給や賞与の仕組
みを理解し、働きがいを持って業務に取り組んでもらうために作られています。
1. 基本給
1. 基本給とは
基本給は、みらいテクノロジー株式会社で働くすべての従業員に⽀払われる基本的
な賃⾦です。
基本給は、従業員の経験や能⼒、職務内容に基づいて決定されます。
2. 基本給の決定⽅法
基本給は、毎年の⼈事評価結果や会社の業績、個⼈の勤務年数などを考慮して⾒直
しが⾏われる場合があります。
新⼊社員の場合は、会社の定める給与テーブルに基づき、職種ごとに⼀定の基準額
が設定されています。
2. 各種⼿当
みらいテクノロジー株式会社では、基本給に加えて、以下の⼿当が⽀給されます。
1. 通勤⼿当
通勤にかかる交通費は、実際の経路に基づき⽀給されます。
公共交通機関の利⽤の場合は、最安経路をもとに⽉額上限 3 万円まで⽀給します。
⾃家⽤⾞での通勤が必要な場合は、事前に⼈事部へ申請してください。駐⾞場の使
⽤料やガソリン代の⼀部が⽀給される場合もあります。
2. 住宅⼿当
会社から通勤に 1 時間以上かかる場合、住宅⼿当として⽉額 1 万円が⽀給されます。
住宅⼿当を受けるためには、賃貸契約書など、居住地を証明できる書類の提出が必
要です。
3. 家族⼿当
扶養家族がいる従業員には、家族⼿当が⽀給されます。
配偶者には⽉額 5,000 円、⼦供⼀⼈につき⽉額 3,000 円が⽀給されます（上限︓⼦供 3
⼈まで）。
4. 時間外⼿当（残業代）
所定の勤務時間を超えて働いた場合は、残業⼿当が⽀給されます。
残業⼿当の割増率は、法令に基づき計算されます。通常の時間外労働は 1.25 倍、深
夜時間帯（午後 10 時以降）の残業は 1.5 倍となります。
5. 休⽇出勤⼿当' metadata={'source': 'data/pdf/02賃金規則.pdf', 'page': 0}
--------------------------------------------------
page_content='休

In [ ]:
# インデックスの構築
"""
チャンクをもとにインデックスを構築します。ベクトルDBとして今回はChromaを使用しています。
エンベディングモデルには、OpenAIの "text-embedding-3-small" モデルを指定しています。
なお、Failed to send telemetry event ClientStartEvent...というメッセージが表示されても無視して進めて頂いて大丈夫です。
"""
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# エンベディングモデルの指定
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# インデックスの構築
db = Chroma.from_documents(texts, embedding_model)
print("インデックスの構築が完了しました") # デバッグ用
db # デバッグ用

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


インデックスの構築が完了しました


In [8]:
# 動作確認
"""インデックスから検索が行えるか確認しましょう。検索には Retriever コンポーネントを使用します。"""
# Retrieverの作成
retriever = db.as_retriever()

# 検索の実施
results = retriever.invoke("有給休暇の付与日数は？")

# 結果を表示
for result in results:
    print(result.page_content)
    print("-" * 50)

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


みらいテクノロジー株式会社  休暇規則
みらいテクノロジー株式会社では、従業員の皆さんが仕事と⽣活のバランスを保ちながら働ける
よう、さまざまな休暇制度を設けています。この休暇規則は、休暇の種類や取得⽅法、条件など
を明確にし、安⼼して休暇を利⽤していただくためのものです。
1. 年次有給休暇（有給休暇）
1. 有給休暇とは
有給休暇は、給与を受け取りながら休暇を取得できる制度です。
⼼⾝のリフレッシュや私⽤のために⾃由に利⽤できます。
2. 付与⽇数
⼊社から 6 ヶ⽉継続勤務し、全労働⽇の 8 割以上出勤した場合に、初めて有給休暇が
付与されます。
初年度は 10 ⽇間の有給休暇が付与され、その後は勤続年数に応じて増加します。
勤続年数年次有給休暇⽇数
0.5 年 10 ⽇
1.5 年 11 ⽇
2.5 年 12 ⽇
3.5 年 14 ⽇
4.5 年 16 ⽇
5.5 年 18 ⽇
6.5 年以上20 ⽇
3. 有給休暇の取得⽅法
有給休暇を取得する際は、原則として3 ⽇前までに上司に申請してください。
緊急の場合は、当⽇の申請も可能ですが、できるだけ早めに連絡をお願いします。
申請は、社内の休暇申請システムを利⽤してください。
4. 有給休暇の繰越し
未使⽤の有給休暇は、翌年度に限り繰り越すことができます。
最⼤で 40 ⽇間の有給休暇を保有することが可能です。
2. 特別休暇
特別休暇は、有給休暇とは別に特定の事情に応じて取得できる休暇です。
1. 慶弔休暇
結婚休暇︓本⼈が結婚する場合、5 ⽇間の休暇が取得できます。
--------------------------------------------------
みらいテクノロジー株式会社  休暇規則
みらいテクノロジー株式会社では、従業員の皆さんが仕事と⽣活のバランスを保ちながら働ける
よう、さまざまな休暇制度を設けています。この休暇規則は、休暇の種類や取得⽅法、条件など
を明確にし、安⼼して休暇を利⽤していただくためのものです。
1. 年次有給休暇（有給休暇）
1. 有給休暇とは
有給休暇は、給与を受け取りながら休暇を取得できる制度です。
⼼⾝のリフレッシュや私⽤のために⾃由に利⽤できます。
2. 付与⽇数
⼊社から 6 ヶ⽉継続勤務し、全労働⽇の 8 割以上出勤した場合に、初めて有給休暇が
付与

In [9]:
## 参考：インデックスの保存と読込

# 保存先を指定
"""インデックスをストレージに保存するには persist_directory オプションで保存先を指定します。"""
db = Chroma.from_documents(texts, embedding_model, persist_directory="./chroma_db")

# ストレージから復元
"""ストレージから復元するには、persist_directory オプションを指定してChromaオブジェクトを作成します。"""
db = Chroma(persist_directory="./chroma_db", embedding_function=embedding_model)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [14]:
## 4.3 RAGチェーンの作成
"""
プロンプトテンプレートと言語モデルを用意して、RAGチェーンを作成しましょう。以下のようなチェーンを作成します。
Chapter 4.3 RAGチェーンの作成は、分割して掲載されていたので、まとめた。
"""

from langchain_core.prompts import ChatPromptTemplate

# プロンプトテンプレートの作成
"""プロンプトテンプレートを作成します。"""
prompt = ChatPromptTemplate.from_template("""提供されたコンテキストのみに基づいて次の質問に答えてください:

<コンテキスト>
{context}
</コンテキスト>

Question: {input}""")

# 言語モデルの作成
"""言語モデルコンポーネントを作成します"""
# モデルの作成
chat_model = ChatOpenAI(model_name=MODEL_NAME)

# チェーンの作成
"""Retriever、プロンプトテンプレート、言語モデルをもとにチェーンを作成します。"""
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# チェーンを構成する要素
"""
>> {"context": retriever, "input": RunnablePassthrough()}
複数のコンポーネントを並行して実行するには {キー名:コンポーネント, キー名:コンポーネント} と指定します。RunnablePassthrough() は入力をそのまま出力するコンポーネントです。
・入力：ユーザーからの質問（invoke で与えられる）
・出力：要素が各キー名の辞書。{"context": retrieverの結果, "input": ユーザーからの質問}

>> prompt
プロンプトテンプレートです。テンプレートは {context} と {input} を受け取ります。これは入力と一致しています。
・入力：{"context": retrieverの結果, "input": ユーザーからの質問}
・出力：ChatPromptTemplate 型のプロンプト

>> chat_model
言語モデルです。
・入力：ChatPromptTemplate 型のプロンプト
・出力：AIMessage 型の言語モデルからの回答

>> StrOutputParser()
AIMessage 型などの回答から、メッセージの文字列を取り出します。
・入力：AIMessage 型の言語モデルからの回答
・出力：メッセージ（文字列）
"""
chain = ({"context": retriever, "input": RunnablePassthrough()}
    | prompt
    | chat_model
    | StrOutputParser())

# チェーンの実行
response = chain.invoke("有給休暇の付与日数は？")

# 結果を表示
print(response)

有給休暇の付与日数は、勤続年数に応じて以下のようになります：

- 0.5 年：10 日
- 1.5 年：11 日
- 2.5 年：12 日
- 3.5 年：14 日
- 4.5 年：16 日
- 5.5 年：18 日
- 6.5 年以上：20 日

最初の有給休暇は、入社から6ヶ月継続勤務し、全労働日の8割以上出勤した場合に付与されます。初年度は10日間の有給休暇が付与され、その後は勤続年数に応じて増加します。
